## 텍스트 분류 실습 - 20 뉴스그룹 분류

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset="all", random_state=156)

In [3]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [4]:
import pandas as pd

print("target 클래스의 값과 분포도 \n", pd.Series(news_data.target).value_counts().sort_index())
print("target 클래스의 이름들\n", news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
Name: count, dtype: int64
target 클래스의 이름들
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [7]:
from sklearn.datasets import fetch_20newsgroups


# subset="train"으로 학습용 데이터만 추출
# remove = ("headers","footers",quotes)로 내용만 추출

train_news = fetch_20newsgroups(subset="train", remove=("headers","footers","quotes"),
                                random_state=156)

X_train = train_news.data
y_train = train_news.target

In [8]:
test_news = fetch_20newsgroups(subset="test", remove=("headers","footers","quotes"),
                                random_state=156)

X_test = test_news.data
y_test = test_news.target

In [9]:
print("학습 데이터 크기 {0}, 텍스트 데이터 크기 {1}".format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 텍스트 데이터 크기 7532


#### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 텍스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)


print("학습 데이터 텍스트의 CountVectorizer Shape:", X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


> 유의할 점은 테스트 데이터에서 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환(transform)해야 한다는 것. 그래야만 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아짐. 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform()을 이용해 변환 

> 11314개의 문서에서 피처, 즉 단어가 101631개로 만들어짐 

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행. 
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)

print("CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.606


/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


> Count 기반에서 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행 <br>
TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공 <br>
일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print("TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


> 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것. 텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있음 

In [17]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용.

tfidf_vect = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)

print("TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


In [18]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정.
params = {"C":[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring="accuracy", verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print("Logistic Regression best C parameter :", grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print("TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


In [19]:
from sklearn.pipeline import Pipeline

#TFidVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline생성
pipeline = Pipeline([
    ("tfidf_vect", TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_df=300)),
    ("lr_clf", LogisticRegression(C=10))
])

#별도의 TFidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음
#pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능

pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)

print("Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


In [24]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ("tfidf_vect", TfidfVectorizer(stop_words="english")),
    ("lr_clf", LogisticRegression())
])

params = {"tfidf_vect__ngram_range":[(1,1),(1,2),(1,3)],
         "tfidf_vect__max_df":[100,300,700],
         "lr_clf__C":[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline,param_grid=params, cv=3, scoring="accuracy",verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_,grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print("Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}".format(accuracy_score(y_test,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

{'lr_clf__C': 10, 'tfidf_vect__max_df': 300, 'tfidf_vect__ngram_range': (1, 2)} 0.7536687914006531
Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


# 감성 분석 

: 감성 분석은 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용되고 있습니다. 감성분석은 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성 수치를 계산하는 방법을 이용합니다. 이러한 감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성되며 이들 지수를 합산해 긍정 감성 또는 부정 감성을 결정합니다 

- 지도학습 <br>
학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일합니다

- 비지도학습 <br>
"Lexicon"이라는 일종의 감성 어휘 사전을 이용합니다. Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며, 이를 이ㅛㅇ해 문서의 긍정적, 부정적 감성 여부를 판단합니다

## 지도학습 기반 감성 분석

In [25]:
import pandas as pd

review_df = pd.read_csv("./word2vec-nlp-tutorial/labeledTrainData.tsv", 
                       header=0, sep="\t", quoting=3)

review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [26]:
print(review_df["review"][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [27]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df["review"]=review_df["review"].str.replace("<br />"," ")

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df["review"]=review_df["review"].apply(lambda x : re.sub("[^a-zA-Z]"," ",x))

In [28]:
from sklearn.model_selection import train_test_split

class_df = review_df["sentiment"]
feature_df = review_df.drop(["id","sentiment"], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3,
                                                   random_state=156)


In [29]:
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [33]:
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization 수행.
# LogisticRegression의 C는 10으로 설정

pipeline = Pipeline([
    ("cnt_vect", CountVectorizer(stop_words="english", ngram_range=(1,2))),
    ("lr_clf", LogisticRegression(C=10))
])

In [35]:
# Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc 때문에 수행

pipeline.fit(X_train["review"],y_train)
pred = pipeline.predict(X_test["review"])
pred_probas = pipeline.predict_proba(X_test["review"])[:,1]

print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}".format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probas)))

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


예측 정확도는 0.8860, ROC-AUC는 0.9503


In [36]:
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행.
# LogisticRegression의 C는 10으로 설정

pipeline = Pipeline([
    ("tfidf_vect", TfidfVectorizer(stop_words="english", ngram_range=(1,2))),
    ("lr_clf", LogisticRegression(C=10))
])

In [37]:
pipeline.fit(X_train["review"],y_train)
pred = pipeline.predict(X_test["review"])
pred_probas = pipeline.predict_proba(X_test["review"])[:,1]

print("예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}".format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probas)))

예측 정확도는 0.8936, ROC-AUC는 0.9598


## 비지도학습 기반 감성 분석

: 비지도 감성 분석은 Lexicon을 기반으로 하는 것입니다. 위의 지도 감성 분석은 데이터 세트가 레이블 감ㅅ을 가지고 있었습니다. 하지만 많은 감성 분석용 데이터는 이러한 결정된 레이블 값을 가지고 있지 않습니다. 이러한 경우에 Lexicon은유용하게 사용될 수 있습니다

: Lexicon은 일반적으로 어휘집을 의미하지만 여기서는 주로 감성만을 분석하기 위해 지원하는 감성 어휘 사전입니다. 줄여서 감성 사전으로 표현하겠습니다. 감성 사전은 긍정(Positive) 감성 또는 부정(Negative) 감성 의 정도를 의미하는 수치를 가지고 있으며 이를 감성 지수(Polarity score)라고 합니다. 이 감성 지수는 단어의 위치나 주변 단어, 문맥, POS 등을 참고해 결정됩니다

- 텍스트 분석을 공부하다 보면 아마도 '시맨틱(semantic)'이라는 용어를 자주 접하게 될 것입니다. 시맨틱은 간단히 표현하면 '문맥상 의미'입니다. 알다시피 '말'이라는 것은 상황에 따라, 문맥에 따라, 화자의 몸짓이나 어조에 따라 다르게 해석될 수 있습니다. 동일한 단어나 문장이라도 다른 환경과 문맥에서는 다르게 표현되거나 이해될 수 있습니다.<br> ex) 영어단어 present는 선물이라는 의미도 있지만 현재라는 의미도 있음 <br> ex) "밥 먹었어?"는  단순히 식사했는가를 묻는 표현일 수도 있지만 안부를 묻는 표현일 수도 있음. 
<br>
<br>
- NLP 패키지의 WordNet <br> NLP는 시맨틱을 프로그램적으로 인터페이스할 수 있는 다양한 방법을 제공합니다. WordNet 모듈은 방대한 영어 어휘 사전인데, WordNet은 단순한 어휘 사전이 아닌 시맨틱 분석을 제공하는 어휘 사전입니다. 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시맨틱 정보를 제공하며, 이를 위해 각각의 품사(명사, 동사, 형용사, 부사 등)로 구성된 개별 단어를 Synset(Sets of cognitive synonyms)이라는 개념을 이용해 표현합니다. Synset은 단순한 하나의 단어가 아니라 그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 WordNet의 핵심 개념입니다

### SentiVordNet을 이용한 감성 분석

In [38]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to
[nltk_da

[nltk_data]    |   Unzipping corpora/opinion_lexicon.zip.
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Package panlex_swadesh is already up-to-date!
[nltk_data]    | Downloading package paradigms to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Unzipping corpora/paradigms.zip.
[nltk_data]    | Downloading package pe08 to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Unzipping corpora/pe08.zip.
[nltk_data]    | Downloading package perluniprops to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Unzipping misc/perluniprops.zip.
[nltk_data]    | Downloading package pil to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Unzipping corpora/pil.zip.
[nltk_data]    | Downloading package pl196x to
[nltk_data]    |     /Users/ijiseon/nltk_data...
[nltk_data]    |   Unzipping corpora/pl196x.zip.
[nltk_data]    | Downloading package porter

[nltk_data]    |   Unzipping corpora/ycoe.zip.
[nltk_data]    | 
[nltk_data]  Done downloading collection all


True

- NLTK의 모든 데이터 세트를 내려받은 뒤에 WordNet 모듈을 임포트해서 "present" 단어에 대한 Synset을 추출. WordNet의 synsets()는 파라미터로 지정된 단어에 대해 WordNet에 등재된 모든 Synset 객체를 반환합니다 

In [39]:
from nltk.corpus import wordnet as wn

term ="present"

# "present"라는 단어로 wordnet의 synsets 생성 
synsets = wn.synsets(term)
print("synsets() 반환 type :", type(synsets))
print("synsets() 반환 값 개수:", len(synsets))
print("synsets() 반환 값:", synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 개수: 18
synsets() 반환 값: [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


- synsets() 호출 시 반환되는 것은 여러 개의 Synset 객체를 가지는 리스트입니다. 총 18개의 서로 다른 semantic을 가지는 synset 객체가 반환됐습니다. Synset("present.n.01")와 같이 Synset 객체의 파라미터 "present.n.01"은 POS 태그를 나타냅니다. "present.n.01"에서 present는 의미, n은 명사 품사, 01은 present가 명사로서 가지는 의미가 여러가지 있어서 이를 구분하는 인덱스입니다. 

- synset 객체가 가지는 여러 가지 속성을 살펴보겠습니다. Synset은 POS(Part of Speech로 우리말로 바꾸면 품사입니다). 정의(Definition), 부명제(Lemma) 등으로 시맨틱적인 요소를 표현할 수 있습니다

In [40]:
for synset in synsets:
    print("#### Synset name:", synset.name(),"####")
    print("POS:", synset.lexname())
    print("Definition:", synset.definition())
    print("Lemmas:", synset.lemma_names())

#### Synset name: present.n.01 ####
POS: noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
#### Synset name: present.n.02 ####
POS: noun.possession
Definition: something presented as a gift
Lemmas: ['present']
#### Synset name: present.n.03 ####
POS: noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
#### Synset name: show.v.01 ####
POS: verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
#### Synset name: present.v.02 ####
POS: verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
#### Synset name: stage.v.01 ####
POS: verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'represent']
#### Synset name: present.

- WordNet은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있습니다. synset 객체는 단어 간의 유사도를 나타내기 위해서 path_similarity()메서드를 제공합니다. path_similarity()를 이용해 "tree", "lion", "tiger", "cat", "dog"라는 단어의 상호 유사도를 살펴보겠습니다

In [41]:
# synset 객체를 단어별로 생성합니다.

tree = wn.synset("tree.n.01")
lion = wn.synset("lion.n.01")
tiger = wn.synset("tiger.n.02")
cat = wn.synset("cat.n.01")
dog = wn.synset("dog.n.01")

In [42]:
entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split(".")[0] for entity in entities]

-> 이 코드는 주어진 entities 리스트에서 각 엔티티의 이름을 추출하는 과정을 나타냅니다. 여기서 entities는 WordNet의 단어 네트워크(WordNet)에서 추출한 단어를 나타내는 객체 리스트입니다. 이 리스트에서 각 객체의 이름을 추출하기 위해 리스트 컴프리헨션을 사용합니다.

WordNet의 객체는 'tree.n.01', 'lion.n.01'과 같이 도메인, 품사, 인덱스로 이루어진 텍스트로 표현됩니다. 이 코드에서는 split() 함수를 사용하여 각 객체의 이름을 추출합니다. '.'을 구분자로 사용하여 문자열을 나눈 후 첫 번째 요소를 선택하면 해당 객체의 이름만 남게 됩니다. 따라서 entity.name().split(".")[0]는 객체의 이름만을 추출하는 과정을 의미합니다.

결과적으로 entity_names 리스트에는 각 객체의 이름만이 포함되어 있게 됩니다.

In [45]:
# 단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정합니다. 
for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity),2)
                 for compared_entity in entities]
    similarities.append(similarity)
    

# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame 형태로 저장
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


- SentiWordNet은 WordNet의 Synset과 유사한 Senti_Synset 클래스를 가지고 있습니다. SentiWordNet 모듈의 senti_synsets()는 WordNet 모듈이라서 synsets()와 비슷하게 Senti_Synset 클래스를 리스트 형태로 반환합니다

In [46]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets("slow"))
print("senti_sysets() 반환 type:", type(senti_synsets))
print("senti_synsets() 반환 값 개수:", len(senti_synsets))
print("senti_synsets() 반환 값:", senti_synsets)

senti_sysets() 반환 type: <class 'list'>
senti_synsets() 반환 값 개수: 11
senti_synsets() 반환 값: [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


- SentiSynset 객체는 단어의 감성을 나타내는 감성 지수와 객관성을(감성과 반대) 나타내는 객관성 지수를 가지고 있습니다. 감성 지수는 다시 긍정 감성 지수와 부정 감성 지수로 나뉩니다. 어떤 단어가 전혀 감성적이지 않으면 객관성 지수는 1이 되고, 감성 지수는 모두 0이 됩니다. 다음은 father(아버지)라는 단어와 fabulous(아주 멋진)라는 두 개 단어의 감성 지수와 객관성 지수를 나타냅니다

In [48]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset("father.n.01")
print("father 긍정감성 지수: ", father.pos_score())
print("father 부정감성 지수: ", father.neg_score())
print("father 객관성 지수: ", father.obj_score())
print("--------------------------------------------")
fabulous = swn.senti_synset("fabulous.a.01")
print("fabulous 긍정감성 지수: ", fabulous.pos_score())
print("fabulous 부정감성 지수: ", fabulous.neg_score())
print("fabulous 객관성 지수: ", fabulous.obj_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0
--------------------------------------------
fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125
fabulous 객관성 지수:  0.0


### SentiWordNet을 이용한 영화 감상평 감성 분석 

1. 문서(Document)를 문장(Sentence) 단위로 분해
2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅
3. 품사 태깅된 단어 기반으로 synset 객체와 senti_synset 객체를 생성
4. Senti_synset에서 긍정 감성/ 부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 대는 부정 감성으로 결정

- SentiWordNet을 이용하기 위해서 WordNet을 이용해 문서를 다시 단어로 토큰화한 뒤 어근 추출(Lemmatization)과 품사 태깅(POS Tagging)을 적용해야 합니다. 먼저 품사 태깅을 수행하는 내부 함수를 생성하겠습니다

In [57]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet기반의 품사 Tag로 변환

def penn_to_wn(tag):
    if tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("R"):
        return wn.ADV
    elif tag.startswith("V"):
        return wn.VERB

- 이제 문서를 문장 -> 단어 토큰 -> 품사 태깅 후에 SentiSynset 클래스를 생성하고 Polarity Score를 합산하는 함수를 생성하겠습니다. 각 단어의 긍정 감성 지수와 부정 감성 지수를 모두 합한 총 감성 지수가 0 이상일 경우 긍정 감성, 그렇지 않을 경우 부정 감성으로 예측합니다. 

In [50]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [62]:
def swn_polarity(text):
    # 감성 지수 초기화 
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    
    #분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        #NLTK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:
            
            #WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            #어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            #sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            #모든 단어에 대해 긍정 감성 지수는 +로 부정 감성지수는 -로 합산해 감성 지수 계산 
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score()-swn_synset.neg_score())
            tokens_count += 1 
        
    if not tokens_count:
        return 0
        
    #총 score가 0 이상일 경우 긍정 1, 그렇지 않을 경우 부정 0 반환
    if sentiment>=0:
        return 1
    return 0


> ### 함수 설명 


1. 텍스트 전처리 <br>: 입력 텍스트를 문장으로 분해하고, 각 문장을 단어로 토큰화합니다. 이때 NLTK의 sent_tokenize 함수를 사용하여 문장을 분리하고, word_tokenize 함수를 사용하여 단어를 토큰화합니다.
<br>

2. 품사 태깅<br> : 각 단어에 대해 품사를 태깅합니다. 이는 해당 단어가 명사인지, 형용사인지 등을 판별하기 위함입니다. 이 과정에서 NLTK의 pos_tag 함수를 사용합니다. <br><br>-> tagged sentence는 각 단어와 해당 단어의 품사 태그로 이루어진 튜플의 리스트입니다. 각 튜플은 단어와 그 단어의 품사 태그로 구성되어 있습니다. <br> ex)예를 들어, "I love cats"라는 문장을 토큰화하고 품사 태깅한 결과는 다음과 같을 수 있습니다 [('I', 'PRP'), ('love', 'VBP'), ('cats', 'NNS')]
<br>

3. WordNet을 이용한 품사 태깅과 어근 추출<br>: 품사 태깅된 단어를 기반으로 WordNet을 활용하여 어근을 추출합니다. 이때 각 단어의 품사 태그를 바탕으로 적절한 품사 태그를 지정하여 어근을 추출합니다.<br><br> -> penn_to_wn(tag) 함수는 Penn Treebank POS 태그를 WordNet 품사 태그로 변환합니다. <br> 여기서 tag는 주어진 단어의 Penn Treebank POS 태그입니다. 이것은 영어 단어의 품사를 나타내는데 사용됩니다. 예를 들어, 'JJ'는 형용사(Adjective)를 나타냅니다.penn_to_wn 함수는 주어진 Penn Treebank POS 태그를 WordNet에서 사용하는 품사 태그로 변환하여 반환합니다. 그래서 wn_tag는 WordNet에서 해당 단어를 사용하기 위해 사용할 수 있는 품사 태그가 됩니다.예를 들어, 'JJ' (형용사)는 WordNet에서 'ADJ'로 변환됩니다. 이렇게 변환된 품사 태그는 WordNet에서 해당 단어와 관련된 시소러스 정보를 찾을 때 사용됩니다.
<br>

4. SentiWordNet을 이용한 감성 분석 <br>: 추출된 어근을 바탕으로 해당 단어가 감성적인지, 그리고 긍정적인지 혹은 부정적인지를 판단합니다. 이를 위해 SentiWordNet을 활용합니다. SentiWordNet은 각 단어의 감성 점수를 제공하여 해당 단어가 갖는 긍정 감성과 부정 감성을 알려줍니다.
<br>

5. 감성 지수 계산 <br>: 각 단어의 감성 점수를 합산하여 전체 문장의 감성 지수를 계산합니다. 이때 긍정 감성과 부정 감성을 각각 더하고 뺀 후 이를 합산합니다.
<br>

6. 결과 반환 <br>: 계산된 감성 지수를 바탕으로 해당 텍스트가 긍정적인지 아니면 부정적인지를 판단합니다. 감성 지수가 0 이상이면 긍정적으로 판단하고, 그렇지 않으면 부정적으로 판단합니다.

- if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):<br>: 단어의 품사가 명사, 형용사 또는 부사가 아닌 경우, 즉 주어진 분류 중에 없는 경우, 다음 단계로 넘어갑니다. <br> 다음 단계는 해당 단어가 감성 분석에 유효한 단어로 간주되지 않는 경우입니다. 따라서 다음 단계로 넘어가지 않고 그냥 반복문을 계속 진행하게 됩니다. 코드에서는 continue 문을 사용하여 반복문의 다음 반복 단계로 이동합니다. 이것은 해당 단어가 유효한 단어가 아니므로 처리를 건너뛰고 다음 단어로 넘어가야 함을 의미합니다.
- lemma = lemmatizer.lemmatize(word, pos=wn_tag)<br>: 해당 단어의 어근을 추출합니다. 어근 추출은 단어의 기본 형태를 찾는 과정으로, 'running'의 어근은 'run', 'better'의 어근은 'good' 등으로 변환됩니다.
- synsets = wn.synsets(lemma, pos=wn_tag)<br>: WordNet에서 해당 단어의 동의어 집합(synset)을 찾습니다. 이는 주어진 단어에 대한 시소러스 정보를 제공하며, 단어의 의미와 관련된 다양한 정보를 포함합니다.
- synset = synsets[0]:<br> 단어의 동의어 집합 중 첫 번째를 선택합니다. 이는 단어의 가장 일반적인 의미를 나타냅니다.
- swn_synset = swn.senti_synset(synset.name()):<br>: SentiWordNet에서 해당 동의어 집합에 대한 감성 정보를 가져옵니다. SentiWordNet은 감성 단어 분석을 위한 WordNet의 확장입니다.
- sentiment += (swn_synset.pos_score()-swn_synset.neg_score()):<br>: 해당 단어의 긍정 감성 지수와 부정 감성 지수를 계산하여 감성 지수를 업데이트합니다.
- tokens_count += 1:<br>: 처리된 토큰(단어)의 수를 증가시킵니다. 이것은 후에 감성 지수를 평균화하기 위해 사용됩니다.

In [63]:
review_df.head(10)

,id,sentiment,review
0,"""5814_8""",1,With all this stuff going down at the moment ...
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...
3,"""3630_4""",0,It must be assumed that those who praised thi...
4,"""9495_8""",1,Superbly trashy and wondrously unpretentious ...
5,"""8196_8""",1,I dont know why people think this is such a b...
6,"""7166_2""",0,This movie could have been very good but com...
7,"""10633_1""",0,I watched this video at a friend s house I m...
8,"""319_1""",0,A friend of mine bought this film for and...
9,"""8713_10""",1,This movie is full of references Like Ma...


In [64]:
review_df["preds"]=review_df["review"].apply(lambda x : swn_polarity(x))
y_target = review_df["sentiment"].values
preds = review_df["preds"].values

In [68]:
review_df.head(10)

,id,sentiment,review,preds
0,"""5814_8""",1,With all this stuff going down at the moment ...,0
1,"""2381_9""",1,The Classic War of the Worlds by Timothy ...,1
2,"""7759_3""",0,The film starts with a manager Nicholas Bell...,0
3,"""3630_4""",0,It must be assumed that those who praised thi...,0
4,"""9495_8""",1,Superbly trashy and wondrously unpretentious ...,0
5,"""8196_8""",1,I dont know why people think this is such a b...,1
6,"""7166_2""",0,This movie could have been very good but com...,0
7,"""10633_1""",0,I watched this video at a friend s house I m...,0
8,"""319_1""",0,A friend of mine bought this film for and...,0
9,"""8713_10""",1,This movie is full of references Like Ma...,1


In [66]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target,preds))
print("정확도:",np.round(accuracy_score(y_target,preds),4))
print("정밀도:",np.round(precision_score(y_target,preds),4))
print("재현율:",np.round(recall_score(y_target,preds),4))

[[7668 4832]
 [3636 8864]]
정확도: 0.6613
정밀도: 0.6472
재현율: 0.7091


### VADER를 이용한 감성 분석

In [69]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df["review"][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


VADER를 이용하면 매우 쉽게 감성 분석을 수행할 수 있습니다. 먼저 SentimentIntensityAnalyzer 객체를 생성한 뒤에 문서별로 polarity_scores()메서드를 호출해 감성 점수를 구한 뒤, 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단합니다. 

Analyzer 객체의 polarity_scores()메서드는 딕셔너리 형태의 감성 점수를 반환합니다. "neg"는 부정 감성 지수, "neu"는 중립적인 감성 지수, "pos"는 긍정적인 감성 지수, 그리고 compound는 neg, neu, pos score를 적절히 조합해 -1에서 1 사이의 감성 지수를 표현한 값입니다. compound score를 기반으로 부정 감성 또는 긍정 감성 여부를 결정합니다. 보통 0.1 이상이면 긍정 감성, 그 이하이면 부정 감성으로 판단하나 상황에 따라 이 임계값을 적절히 조정해 예측 성능을 조절합니다

In [73]:
def vader_polarity(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    #compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
    agg_score = scores["compound"]
    final_sentiment = 1 if agg_score >= threshold else 0 
    return final_sentiment

In [74]:
# apply lambda 식을 이용해 레코드 별로 vader_polarity()를 수행하고 결과를 "vader_preds"에 저장

review_df["vader_preds"]=review_df["review"].apply(lambda x : vader_polarity(x,0.1))
y_target = review_df["sentiment"].values
vader_preds = review_df["vader_preds"].values

print(confusion_matrix(y_target,vader_preds))
print("정확도:",np.round(accuracy_score(y_target,vader_preds),4))
print("정밀도:",np.round(precision_score(y_target,vader_preds),4))
print("재현율:",np.round(recall_score(y_target,vader_preds),4))

[[ 6747  5753]
 [ 1858 10642]]
정확도: 0.6956
정밀도: 0.6491
재현율: 0.8514
